# Segformer

Implementation adapted from: https://github.com/NVlabs/SegFormer

Research Paper: https://arxiv.org/abs/2105.15203

Datasets: https://data.mendeley.com/datasets/8gf9vpkhgy/2

In [2]:
import os
import numpy as np
import torch
import glob
import cv2
from tqdm import tqdm
from torch import nn
from transformers import SegformerConfig, SegformerModel, TrainingArguments, AutoImageProcessor, SegformerForSemanticSegmentation
from transformers import Trainer
from einops import rearrange
from sklearn.model_selection import train_test_split
import evaluate

c:\Users\duong\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Section 1: Darwin Dataset

This section will go through the implementation of Segformer using the Darwin dataset

### Part 1: Prepare Dataset

Implementation adapted from: https://www.youtube.com/watch?v=-AejMcdeOOM

In [3]:
Height = 256
Width = 256

path = "./Datasets/Darwin/"
imagePath = path + "img/*.png"
maskPath = path + "img/*.png"

listOfImages = glob.glob(imagePath)
listofMaskImages = glob.glob(maskPath)

print("Images in Folder:", len(listOfImages))
print("Mask Images in Folder:", len(listofMaskImages))

Images in Folder: 6106
Mask Images in Folder: 6106


In [4]:
img = []
mask = []

for imgFiles, maskImgFiles in tqdm(zip(listOfImages, listofMaskImages), total=len(listOfImages)):
    images = cv2.imread(imgFiles, cv2.IMREAD_COLOR)
    images = cv2.resize(images, (Width, Height))
    images = images / 255.0
    images = images.astype(np.float32)
    img.append(images)

    maskImages = cv2.imread(maskImgFiles, cv2.IMREAD_GRAYSCALE)
    maskImages = cv2.resize(maskImages, (Width, Height))
    maskImages[maskImages>0] = 1
    mask.append(maskImages)

img = np.array(img)
mask = np.array(mask)
mask = mask.astype(int)

train_imgs, val_imgs = train_test_split(img, test_size=0.1, random_state=10)
train_mask, val_mask = train_test_split(mask, test_size=0.1, random_state=10)

100%|██████████| 6106/6106 [01:38<00:00, 62.06it/s]


### Part 2: Model Implementation

Implemenation adapted from: https://github.com/FrancescoSaverioZuppichini/SegFormer

https://github.com/huggingface/blog/blob/main/fine-tune-segformer.md

In [5]:
processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
c:\Users\duong\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\segformer\image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


### Part 3: Model Evaluation